In [1]:
import pandas as pd
import numpy as np

In [2]:
data = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv"
!wget $data

--2023-09-25 17:10:32--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... failed: Network is unreachable.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... failed: Network is unreachable.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... failed: Network is unreachable.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... failed: Network is unreachable.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===============

In [3]:
initial_df = pd.read_csv("housing.csv")
initial_df.shape

(20640, 10)

In [4]:
sorted(initial_df)

['households',
 'housing_median_age',
 'latitude',
 'longitude',
 'median_house_value',
 'median_income',
 'ocean_proximity',
 'population',
 'total_bedrooms',
 'total_rooms']

## DataPrep

In [5]:
list_proximity = [
    '<1H OCEAN',
    'INLAND',
]

cols = [
    'latitude',
    'longitude',
    'housing_median_age',
    'total_rooms',
    'total_bedrooms',
    'population',
    'households',
    'median_income',
    'median_house_value'
    ]

df = initial_df.query("ocean_proximity.isin(@list_proximity)").loc[:, cols]

# Question 1
There's one feature with missing values. What is it?

In [6]:
df.isnull().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

In [7]:
print("The column total_bedrooms contains missing values.")

The column total_bedrooms contains missing values.


# Question 2
What's the median (50% percentile) for variable 'population'?

In [8]:
population_median = df["population"].median()
print(f"The median fot variable population is {population_median:.0f}.")

The median fot variable population is 1195.


# Additional Steps

Prepare and split the dataset:

* Shuffle the dataset (the filtered one you created above), use seed 42.

In [9]:
np.random.seed(42)
n = df.shape[0]
idx = np.arange(n)
np.random.shuffle(idx)

* Split your data in train/val/test sets, with 60%/20%/20% distribution.

In [10]:
n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test

# Validation
n_val + n_test + n_train == n

True

In [11]:
df_train = df.iloc[idx[:n_train]].reset_index(drop=True)
df_val = df.iloc[idx[n_train:n_train+n_val]].reset_index(drop=True)
df_test = df.iloc[idx[n_train+n_val:]].reset_index(drop=True)

# Validation
df_train.shape[0] + df_val.shape[0] + df_test.shape[0] == df.shape[0]

True

* Apply the log transformation to the **median_house_value** variable using the np.log1p() function.

In [12]:
y_train = np.log1p(df_train["median_house_value"].values)
y_val = np.log1p(df_val["median_house_value"].values)
y_test = np.log1p(df_test["median_house_value"].values)

del df_train['median_house_value'], df_val['median_house_value'], df_test['median_house_value']

# Question 3
We need to deal with missing values for the column from Q1.

We have two options: fill it with 0 or with the mean of this variable.

Try both options. For each, train a linear regression model without regularization using the code from the lessons.

For computing the mean, use the training only!

Use the validation dataset to evaluate the models and compare the RMSE of each option.

Round the RMSE scores to 2 decimal digits using round(score, 2)

Which option gives better RMSE?

In [13]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    
    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]


def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [14]:
df_train.isnull().sum()

latitude               0
longitude              0
housing_median_age     0
total_rooms            0
total_bedrooms        94
population             0
households             0
median_income          0
dtype: int64

## 1º Option: Zero values.

In [15]:
X_train = df_train.fillna(0).values

w_0, w = train_linear_regression(X_train, y_train)

In [16]:
y_pred_train = w_0 + X_train.dot(w)

In [17]:
X_val = df_val.fillna(0).values
y_pred_val = w_0 + X_val.dot(w)

In [18]:
rmse_val0 = round(rmse(y_val, y_pred_val), 2)
rmse_val0

0.34

In [19]:
del X_train, w_0, w, y_pred_train, X_val, y_pred_val

## 2º Option: Mean value.

In [20]:
mean_values = df_train.mean()
X_train = df_train.fillna(mean_values).values

In [21]:
w_0, w = train_linear_regression(X_train, y_train)

In [22]:
y_pred_train = w_0 + X_train.dot(w)

In [23]:
X_val = df_val.fillna(mean_values).values
y_pred_val = w_0 + X_val.dot(w)

In [24]:
rmse_val1 = round(rmse(y_val, y_pred_val), 2)
rmse_val1

0.34

In [25]:
rmse_val0, rmse_val1

(0.34, 0.34)

In [26]:
print("Both options are equally good.")

Both options are equally good.


# Question 4
Now let's train a regularized linear regression.

For this question, fill the NAs with 0.

Try different values of r from this list: [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10].

Use RMSE to evaluate the model on the validation dataset.

Round the RMSE scores to 2 decimal digits.

Which r gives the best RMSE?

If there are multiple options, select the smallest r.

In [27]:
def train_linear_regression_reg(X, y, r):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [28]:
X_train = df_train.fillna(0).values
X_val = df_val.fillna(0).values

reg_values = [0, 0.000001, 0.001, 0.0001]

In [31]:
def multiple_r_linear_reg(X_train, y_train, y_val, reg_values):
    for r in reg_values:
        w_0, w = train_linear_regression_reg(X_train, y_train, r=r)
        y_pred = w_0 + X_val.dot(w)
        _rmse = rmse(y_val, y_pred)
        print(f"For {r=}, the RMSE is {_rmse:.2f}")

In [32]:
multiple_r_linear_reg(X_train, y_train, y_val, reg_values)

For r=0, the RMSE is 0.34
For r=1e-06, the RMSE is 0.34
For r=0.001, the RMSE is 0.34
For r=0.0001, the RMSE is 0.34


# Question 5
We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.

Try different seed values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].

For each seed, do the train/validation/test split with 60%/20%/20% distribution.

Fill the missing values with 0 and train a model without regularization.

For each seed, evaluate the model on the validation dataset and collect the RMSE scores.

What's the standard deviation of all the scores? To compute the standard deviation, use np.std.

Round the result to 3 decimal digits (round(std, 3))

What's the value of std?

In [35]:
seed_list = np.arange(10)

In [36]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    
    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]


def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

def multiple_runs_linear_regressions(df, seed_list):
    scores = []
    for seed in seed_list:
        np.random.seed(seed)
        n = df.shape[0]
        idx = np.arange(n)
        np.random.shuffle(idx)

        n_val = int(n * 0.2)
        n_test = int(n * 0.2)
        n_train = n - n_val - n_test

        df_train = df.iloc[idx[:n_train]].reset_index(drop=True)
        df_val = df.iloc[idx[n_train:n_train+n_val]].reset_index(drop=True)
        df_test = df.iloc[idx[n_train+n_val:]].reset_index(drop=True)
        
        y_train = np.log1p(df_train["median_house_value"].values)
        y_val = np.log1p(df_val["median_house_value"].values)
        y_test = np.log1p(df_test["median_house_value"].values)

        del df_train['median_house_value'], df_val['median_house_value'], df_test['median_house_value']

        X_train = df_train.fillna(0).values

        w_0, w = train_linear_regression(X_train, y_train)

        X_val = df_val.fillna(0).values
        y_pred_val = w_0 + X_val.dot(w)

        rmse_val = rmse(y_val, y_pred_val)
        scores.append(rmse_val)
    return round(np.std(scores), 3)

In [37]:
std_score = multiple_runs_linear_regressions(df, seed_list)
print(f"The standard deviation is {std_score}, it is low, our model is stable for this problem.")

The standard deviation is 0.005, it is low, our model is stable for this problem.


# Question 6
Split the dataset like previously, use seed 9.

Combine train and validation datasets.

Fill the missing values with 0 and train a model with r=0.001.

What's the RMSE on the test dataset?

In [38]:
def train_linear_regression_reg(X, y, r):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

def multiple_runs_linear_regressions_reg(df, seed_list, r):
    scores = []
    for seed in seed_list:
        np.random.seed(seed)
        n = df.shape[0]
        idx = np.arange(n)
        np.random.shuffle(idx)

        n_val = int(n * 0.2)
        n_test = int(n * 0.2)
        n_train = n - n_val - n_test

        df_train = df.iloc[idx[:n_train]].reset_index(drop=True)
        df_val = df.iloc[idx[n_train:n_train+n_val]].reset_index(drop=True)
        df_test = df.iloc[idx[n_train+n_val:]].reset_index(drop=True)
        
        y_train = np.log1p(df_train["median_house_value"].values)
        y_val = np.log1p(df_val["median_house_value"].values)
        y_test = np.log1p(df_test["median_house_value"].values)

        del df_train['median_house_value'], df_val['median_house_value'], df_test['median_house_value']

        X_train = df_train.fillna(0).values

        w_0, w = train_linear_regression_reg(X_train, y_train, r)

        X_val = df_val.fillna(0).values
        y_pred_val = w_0 + X_val.dot(w)

        rmse_val = rmse(y_val, y_pred_val)
    return np.round(rmse_val, 2)

In [39]:
r_value = [0.001]
seed_value = [9]
rmse_score = multiple_runs_linear_regressions_reg(df, seed_value, r_value)
print(f"The Root Mean Squared Error with {r_value=} is {rmse_score}")

The Root Mean Squared Error with r_value=[0.001] is 0.33
